In [2]:
!pip install xgboost

In [31]:
import sklearn
#doing the same than this web site https://www.kaggle.com/paul92s/linear-svc-classifier
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS


In [164]:
# Pipeline
from sklearn.pipeline import Pipeline

# Classifier imports
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.svm import LinearSVC, SVC

#from bs4 import BeautifulSoup as soup

from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# # Performance metric
from sklearn.metrics import accuracy_score


from spacy.lang.en import English
import spacy
from collections import Counter


import pandas as pd



In [109]:
parser = English()
nlp = spacy.load('en_core_web_lg')

In [140]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [141]:
train.head(20)

,id,description,category
0,1,A marriage of 13 and 18 year old bourbons. A m...,2
1,2,There have been some legendary Bowmores from t...,1
2,3,This bottling celebrates master distiller Park...,2
3,4,What impresses me most is how this whisky evol...,1
4,9,"A caramel-laden fruit bouquet, followed by une...",2
5,12,Distilled at the now legendary Stitzel-Weller ...,2
6,14,"No age statement, but distilled in 1998. A bea...",2
7,15,"Monarch, the 75th anniversary limited edition ...",4
8,17,"The complete package: uncut, unfiltered, full-...",2
9,21,"A marriage of four different bourbons, ranging...",2


In [142]:
test.head(10)

,id,description
0,955,"Think carnival aromas—the good ones, anyway—me..."
1,3532,"A blend of three bourbons, between 6 and 12 ye..."
2,1390,"The nose is focused on cereal, hints of fresh ..."
3,1024,Swiss-based Chapter 7 released this 19 year ol...
4,1902,Valkyrie replaces the current Dark Origins exp...
5,1156,The oldest of Bablair’s current core bottlings...
6,2205,"A sherry butt this time, which has allowed the..."
7,885,"This is one of a trio from Chivas Bros., who e..."
8,1116,"Toasted oak, coffee, cola, molasses, and campf..."
9,3739,"One of a trio from Chivas Bros., who every yea..."


In [217]:
test.iloc[243]

id                                                          4075
description    A generally sweet palate (a bit too sweet, act...
Name: 243, dtype: object

In [144]:
#Cleaning description
#change capitalize to lower
#train['description'] = train['description'].apply(lambda x: x.lower())


In [153]:

def tokenize(doc):
    tokens = parser(doc)
    lemmas = []
    for tok in tokens:
        lemmas.append(tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_)
    tokens = lemmas
    
    # Remove all the Stop Words
    tokens = [tok for tok in tokens if tok not in ENGLISH_STOP_WORDS]

    return tokens

In [146]:
train['description']

0       A marriage of 13 and 18 year old bourbons. A m...
1       There have been some legendary Bowmores from t...
2       This bottling celebrates master distiller Park...
3       What impresses me most is how this whisky evol...
4       A caramel-laden fruit bouquet, followed by une...
5       Distilled at the now legendary Stitzel-Weller ...
6       No age statement, but distilled in 1998. A bea...
7       Monarch, the 75th anniversary limited edition ...
8       The complete package: uncut, unfiltered, full-...
9       A marriage of four different bourbons, ranging...
10      Another excellent Stagg, and considering its a...
11      Sometime recently, the source of this whiskey ...
12      From one barrel, and only sold in one location...
13      Definitely showing its age, but not in a bad w...
14      Unlike the standard “small batch” releases, th...
15      Very similar to last year’s release. (A good t...
16      Perhaps the finest Canadian whisky I have ever...
17      This w

### Split Data

In [147]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(train, test_size=0.2, random_state=42)
print('Training Data:', train.shape)
print('Validation Data:', val.shape)

Training Data: (2068, 3)
Validation Data: (518, 3)


In [148]:
# Split in X, Y
x_train = train['description']
y_train = train['category']
x_val = val['description']
y_val = val['category']
x_test = test['description']

### RandomForestClassifier

In [155]:
vect_RFC = TfidfVectorizer(tokenizer=tokenize, ngram_range=(1,2), min_df=5, max_df=0.5)
clf_RFC = RandomForestClassifier()

In [156]:
pipe_RFC = Pipeline([
                 ('vect', vect_RFC), 
                 ('clf', clf_RFC)
                ])

pipe_RFC.fit(x_train, y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [157]:
prediction_RFC = pipe_RFC.predict(x_val)
print("Accuracy:", accuracy_score(y_val, prediction_RFC))

Accuracy: 0.8436293436293436


### LogisticRegression

In [158]:
vect_LR = TfidfVectorizer(tokenizer=tokenize, ngram_range=(1,2), min_df=5, max_df=0.5)
clf_LR = RandomForestClassifier()

In [159]:
pipe_LR = Pipeline([
                 ('vect', vect_LR), 
                 ('clf', clf_LR )
                ])

pipe_LR.fit(x_train, y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [160]:
prediction_LR = pipe_LR.predict(x_val)
print("Accuracy:", accuracy_score(y_val, prediction_LR))

Accuracy: 0.8223938223938224


### XGBClassifier

In [167]:
vect_XG = TfidfVectorizer(tokenizer=tokenize, ngram_range=(1,2), min_df=5, max_df=0.5)
clf_XG = XGBClassifier()

In [162]:
pipe_XG = Pipeline([
                 ('vect', vect_XG), 
                 ('clf', clf_XG )
                ])

pipe_XG.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
...
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1))])

In [163]:
prediction_XG = pipe_XG.predict(x_val)
print("Accuracy:", accuracy_score(y_val, prediction_XG))

Accuracy: 0.8957528957528957


### SVC

In [168]:
vect_SVC = TfidfVectorizer(tokenizer=tokenize, ngram_range=(1,2), min_df=5, max_df=0.5)
clf_SVC = SVC()

In [169]:
pipe_SVC = Pipeline([
                 ('vect', vect_SVC), 
                 ('clf', clf_SVC )
                ])

pipe_SVC.fit(x_train, y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
...f', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))])

In [170]:
prediction_SVC = pipe_SVC.predict(x_val)
print("Accuracy:", accuracy_score(y_val, prediction_SVC))

Accuracy: 0.6274131274131274


### KNeighborsClassifier

In [172]:
vect_KN = TfidfVectorizer(tokenizer=tokenize, ngram_range=(1,2), min_df=5, max_df=0.5)
clf_KN = KNeighborsClassifier()

In [173]:
pipe_KN = Pipeline([
                 ('vect', vect_KN), 
                 ('clf', clf_KN )
                ])

pipe_KN.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
...ki',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'))])

In [174]:
prediction_KN = pipe_KN.predict(x_val)
print("Accuracy:", accuracy_score(y_val, prediction_KN))

Accuracy: 0.8861003861003861


### GaussianNB NOT useful

In [175]:
vect_GAUSS = TfidfVectorizer(tokenizer=tokenize, ngram_range=(1,2), min_df=5, max_df=0.5)
clf_GAUSS = GaussianNB()

In [178]:
pipe_GAUSS = Pipeline([
                 ('vect', vect_GAUSS), 
                 ('clf', clf_GAUSS )
                ])

pipe_GAUSS.fit(x_train, y_train)

TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.

In [ ]:
prediction_GAUSS = pipe_GAUSS.predict(x_val)
print("Accuracy:", accuracy_score(y_val, prediction_GAUSS))

### MultinomialNB

In [179]:
vect_MB = TfidfVectorizer(tokenizer=tokenize, ngram_range=(1,2), min_df=5, max_df=0.5)
clf_MB = MultinomialNB()

In [180]:
pipe_MB = Pipeline([
                 ('vect', vect_MB), 
                 ('clf', clf_MB )
                ])

pipe_MB.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
...rue,
        vocabulary=None)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [181]:
prediction_MB = pipe_MB.predict(x_val)
print("Accuracy:", accuracy_score(y_val, prediction_MB))

Accuracy: 0.8301158301158301


### BernoulliNB

In [182]:
vect_BNB = TfidfVectorizer(tokenizer=tokenize, ngram_range=(1,2), min_df=5, max_df=0.5)
clf_BNB = BernoulliNB()


In [183]:
pipe_BNB = Pipeline([
                 ('vect', vect_BNB), 
                 ('clf', clf_BNB )
                ])

pipe_BNB.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
... vocabulary=None)), ('clf', BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True))])

In [184]:
prediction_BNB = pipe_BNB.predict(x_val)
print("Accuracy:", accuracy_score(y_val, prediction_BNB))

Accuracy: 0.9285714285714286


### SGDClassifier

In [199]:
from sklearn.linear_model import SGDClassifier

svd = TruncatedSVD(algorithm='randomized', n_iter=150, random_state=42)
vect2 = TfidfVectorizer(stop_words='english')
pipe_lsi = Pipeline([('vect', vect2), ('svd', svd)])
clf = SGDClassifier(n_iter=1500, random_state=42)

pipe_SGDClassifier = Pipeline([('lsi', pipe_lsi), ('clf', clf)])

In [200]:
parameters = {
    'lsi__svd__n_components': (100,200, 300),
    'lsi__vect__max_features': (500,1000),
}

grid_search_SGDClassifier = GridSearchCV(pipe_SGDClassifier,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search_SGDClassifier.fit(x_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  1.1min finished
//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=...dom_state=42, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'lsi__svd__n_components': (100, 200, 300), 'lsi__vect__max_features': (500, 1000)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [201]:
grid_search.best_score_

0.9264990328820116

In [98]:
# Predictions on test sample
prediction_SGDClassifier = grid_search.predict(test['description'])

In [ ]:
# submission = pd.DataFrame({'id': test['id'], 'category':pred})
# submission['category'] = submission['category'].astype('int64')

In [99]:
prediction_SGDClassifier

array([2, 3, 1, 1, 1, 1, 1, 1, 2, 1, 1, 4, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 3, 1, 4, 2, 1, 1, 1, 1, 1, 3, 4, 1, 2, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 4,
       2, 1, 1, 1, 1, 3, 1, 1, 4, 1, 3, 2, 1, 1, 4, 1, 2, 1, 1, 3, 2, 4,
       1, 3, 1, 1, 1, 1, 1, 4, 1, 1, 4, 3, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2,
       3, 1, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1, 2, 1, 1, 1, 1, 2, 2, 4, 1, 1,
       1, 1, 3, 2, 1, 1, 1, 1, 1, 3, 2, 1, 1, 3, 4, 1, 1, 1, 3, 1, 1, 1,
       1, 1, 3, 1, 1, 1, 1, 1, 4, 1, 1, 1, 3, 1, 2, 2, 1, 3, 3, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 4, 1, 3, 1, 4, 1, 1, 2, 3, 1, 1,
       2, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 4, 1, 1, 3, 1, 2, 1, 1, 1, 1, 1,
       1, 4, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 1, 3, 1, 1, 2, 1, 1, 1, 1, 3,
       2, 2, 1, 3, 1, 3, 3, 3, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 3, 1, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 3, 4, 1, 1, 1, 3,
       2, 1])

### SGDC best option of it self!

In [194]:
vect_SGDC = TfidfVectorizer(tokenizer=tokenize, ngram_range=(1,2), min_df=5, max_df=0.5)
clf_SGDC = SGDClassifier(n_iter=300, random_state=42)

In [191]:
pipe_SGDC = Pipeline([
                 ('vect', vect_SGDC), 
                 ('clf', clf_SGDC )
                ])

pipe_SGDC.fit(x_train, y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))])

In [192]:
prediction_SGDC = pipe_SGDC.predict(x_val)
print("Accuracy:", accuracy_score(y_val, prediction_SGDC))

Accuracy: 0.9517374517374517


### This was the best !  LinearSVC

In [149]:
vect = TfidfVectorizer(tokenizer=tokenize, ngram_range=(1,2), min_df=5, max_df=0.5)
clf_LSVC = LinearSVC()

'''
Linear Support Vector Classification.
Similar to SVC with parameter kernel=’linear’, but implemented in terms of liblinear rather than libsvm, so it has more flexibility in the choice of penalties and loss functions and should scale better to large numbers of samples.
This class supports both dense and sparse input and the multiclass support is handled according to a one-vs-the-rest scheme.


Between SVC and LinearSVC: SVC(kernel = 'linear') equivalent LinearSVC()
LinearSVC tends to be faster to converge the larger the number of samples is.

SVC https://towardsdatascience.com/https-medium-com-pupalerushikesh-svm-f4b42800e989
'''

In [150]:
# Define the Pipeline
pipe_LSVC = Pipeline([
                 ('vect', vect), 
                 ('clf', clf_LSVC)
                ])

pipe_LSVC.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [151]:
prediction_LSVC = pipe_LSVC.predict(x_val)
print("Accuracy:", accuracy_score(y_val, prediction_LSVC))

Accuracy: 0.9517374517374517


In [152]:
test_pred_LSVC = pipe_LSVC.predict(x_test)

In [139]:
submission = pd.DataFrame({'id': test['id'], 'category': test_pred_LSVC})
submission.to_csv('submission3.csv', index=0)

In [228]:
from sklearn.calibration import CalibratedClassifierCV

In [233]:
vect_LSVC2 = TfidfVectorizer(tokenizer=tokenize)
clf_LSVC2 = CalibratedClassifierCV(method='sigmoid',  #sigmoid will use Platt's scaling. Refer to documentation for other methods.
                                        cv=5)

In [234]:
pipe_LSVC2 = Pipeline([('vect', vect_LSVC2), ('clf', clf_LSVC2)])

In [235]:
pipe_LSVC2.fit(x_train, y_train)


Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...    vocabulary=None)), ('clf', CalibratedClassifierCV(base_estimator=None, cv=5, method='sigmoid'))])

In [236]:
prediction_LSVC = pipe_LSVC.predict(x_val)
print("Accuracy:", accuracy_score(y_val, prediction_LSVC))


Accuracy: 0.9517374517374517
